# 2 Практическая чаcть


In [35]:
import pandas as pd
import torch
import numpy as np
import scipy 
from collections import defaultdict
import re

## 2.1 Загрузка датасета.

### 1. Cоставьте таблицу, в которой указано число токенов, уникальных токенов, предложений для каждой из трех частей датасета.

In [36]:
folder_name = './filimdb_evaluation/PTB/'
filenames = ['train', 'valid', 'test']

In [37]:
def get_file_info(filename):
    global folder_name 
    tokens_cnt = defaultdict(int)
    cnt_lines = 0
    with open(folder_name + f"ptb.{filename}.txt", 'r') as inp:
        for line in inp:
            cnt_lines += 1
            for token in line.strip().split():
                tokens_cnt[token] += 1
    total_tokens = sum(tokens_cnt.values())
    unique_tokens = len(tokens_cnt.keys())
    return filename, total_tokens, unique_tokens, cnt_lines

data = {
    'file':[],
    'token_cnt':[],
    'unique_tokens':[],
    'sentences_cnt': []
}

for f in filenames:
    f_data = get_file_info(f)
    data['file'].append(f_data[0])
    data['token_cnt'].append(f_data[1])
    data['unique_tokens'].append(f_data[2])
    data['sentences_cnt'].append(f_data[3])
    
df = pd.DataFrame(data=data)
df

,file,token_cnt,unique_tokens,sentences_cnt
0,train,887521,9999,42068
1,valid,70390,6021,3370
2,test,78669,6048,3761


### 2. Приведите 10 самых частотных и 10 самых редких токенов с их частотами.
(тут видимо для всех файлов)

In [38]:
tokens_cnt = defaultdict(int)
for f in filenames:
     with open(folder_name + f"ptb.{f}.txt", 'r') as inp:
        for line in inp:
            for token in line.strip().split():
                tokens_cnt[token] += 1
cnt_list = list(tokens_cnt.items())
cnt_list.sort(key=lambda x: x[1])
most_frequent_data = {'word':[], 'cnt':[]}
for w, c in cnt_list[-10:][::-1]:
    most_frequent_data['word'].append(w)
    most_frequent_data['cnt'].append(c)
least_frequent_data = {'word':[], 'cnt':[]}
for w, c in cnt_list[:10]:
    least_frequent_data['word'].append(w)
    least_frequent_data['cnt'].append(c)

In [39]:
pd.DataFrame(data=most_frequent_data)

,word,cnt
0,the,59421
1,<unk>,53299
2,N,37607
3,of,28427
4,to,27430
5,a,24755
6,in,21032
7,and,20404
8,'s,11555
9,for,10436


In [40]:
pd.DataFrame(data=least_frequent_data)

,word,cnt
0,buffet,5
1,lancaster,5
2,barnett,5
3,rewrite,5
4,downgrading,5
5,backgrounds,5
6,stanza,5
7,vessel,5
8,unstable,5
9,peat,5


### 3. Какие специальные токены уже есть в выборке, что они означают?


Вроде как, токены выглядят как текст в треугольных кавычках. Поищем такие фрагменты.

In [41]:
spec_tokens = set()
for f in filenames:
     with open(folder_name + f"ptb.{f}.txt", 'r') as inp:
        for line in inp:
            cur_spec = set(re.findall(r'<[a-z]*>', line))
            spec_tokens = spec_tokens.union(cur_spec)
print(spec_tokens)

{'<unk>'}


Получается, что либо я перепутал тип токенов, либо у нас есть только один специальный токен : $<unk>$

Прочитав документацию по датасету, можно понять, что в нём содержатся 10000 самых популярных токенов, а все остальные токены заменяются на $<unk>$

## 2.2 Генерацей батчей.

In [42]:
def print_batch(ind, X_b, Y_b):
    print(f"Batch # {ind}")
    for i in range(len(X_b)):
            print(X_b[i], ' ', Y_b[i])

In [52]:
def batch_generator(data_path, batch_size, num_steps, debug=False):
    eos_token = '<eos>'
    L_tokens = []
    with open(data_path, 'r', encoding='utf-8') as inp:
        for line in inp:
            line_tokens = list(map(str.lower, line.strip().split()))
            L_tokens.extend(line_tokens)
    L_shifted = L_tokens[1:] + [eos_token]    
    
    unk_len = len(L_tokens) // batch_size
    X_lists = [L_tokens[i : i + unk_len] for i in range(0, len(L_tokens), unk_len) if len(L_tokens[i : i + unk_len]) == unk_len]
    Y_lists = [L_shifted[i : i + unk_len] for i in range(0, len(L_shifted), unk_len)  if len(L_shifted[i : i + unk_len]) == unk_len] 
       
    for i in range(0, unk_len, num_steps):
        
        X_batch = []
        Y_batch = []
        for lst in X_lists:
            X_batch.append(lst[i : i + num_steps])
        for lst in Y_lists:
            Y_batch.append(lst[i : i + num_steps])
        if debug:
            print_batch(i // num_steps, X_batch, Y_batch)
    
batch_generator(folder_name + "small.txt", batch_size = 2, num_steps = 3, debug=True)

Batch # 0
['мороз', 'и', 'солнце']   ['и', 'солнце', 'день']
['ты', 'дремлешь', 'друг']   ['дремлешь', 'друг', 'прелестный']
Batch # 1
['день', 'чудесный', 'еще']   ['чудесный', 'еще', 'ты']
['прелестный', 'пора', 'красавица']   ['пора', 'красавица', 'проснись']


На файле из первых трех строчек **train** датасета функция создаёт батчи похожие на правду.

## 2.3 Реализация LSTM LM.

### 2.3.1 Класс LSTMCell
Для реализации LSTM ячейки будем отталкиваться от реализации обычной RNN ячейки из семинара.

In [47]:
class LSTMCell(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        '''
        Args:
            input_size: Size of token embedding
            hidden_size: Size of hidden state of LSTM cell
        '''
        super(RNNCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Creating matrices whose weights will be trained
        # Token embedding (input of this cell) will be multiplied by this matrix
        self.U_input = nn.Parameter(torch.Tensor(input_size, 4 * hidden_size))
        self.BU_input = nn.Parameter(torch.Tensor(4 * hidden_size))

        # Creating matrices whose weights will be trained
        # Hidden state from previous step will be multipied by this matrix
        # Zero hidden state at the initial step
        self.W_hidden = nn.Parameter(torch.Tensor(hidden_size, 4 * hidden_size))
        self.BW_hidden = nn.Parameter(torch.Tensor(4 * hidden_size))

        # Weights initialization
        self.reset_parameters()

    def forward(self, inp: torch.Tensor, cell_state: torch.Tensor, hidden_state: torch.Tensor) -> (torch.Tensor, torch.Tensor):
        '''
        Performes forward pass of the recurrent cell
        Args:
            inp: Output from Embedding layer at the current timestep
                Tensor shape is (batch_size, emb_size)
            cell_state: Output cell_state from previous recurrent step or zero state
                Tensor shape is (batch_size, hidden_size)
            hidden_state: Output hidden_state from previous recurrent step or zero state
                Tensor shape is (batch_size, hidden_size)
        Returns:
            Output from LSTM cell
        '''
        hidden_mult = hidden_state @ W_hidden + BW_hidden
        input_mult  = inp @ U_input + BU_input 
        matr_sum = input_mult + hidden_mult
        
        sum_chunk = matr_sum.chunk(chunks=4, dim=1)
        f, i, o = torch.sigmoid(sum_chunk[0]), torch.sigmoid(sum_chunk[1]), torch.sigmoid(sum_chunk[2])
        c_new = torch.tanh(sum_chunk[3])
        
        cell_state_new = cell_state * f + i * c_new
        hidden_state_new = o * torch.tanh(cell_state_new)
        
        return cell_state_new, hidden_state_new
        
    def reset_parameters(self):
        '''
        Weights initialization
        '''
        stdv = 1.0 / np.sqrt(self.hidden_size)
        for weight in self.parameters():
            nn.init.uniform_(weight, -stdv, stdv)

8 матриц и векторов смещений заменили на 2 каждого вида.<br>
Всё перемножили и сложили по формулам, применили функции активация к каждой из 4 частей большой матрицы. <br>
Дальше осталось просто всё правильно поэлементно перемножить и получить новые состояния ячейки и скрытое состояние.

### 2.3.2 Класс LSTMLayer

In [53]:
class LSTMLayer(torch.nn.Module):
    def __init__(self, emb_size, hidden_size):
        self.input_size = emb_size
        self.hidden_size = hidden_size
        self.LSTMCell = LSTMCell(emb_size, hidden_size)
        
    def forward(self, X_batch, initial_states=None):
        if initial_states is None:
            cell_state = np.zeros((X_batch.shape[0], self.hidden_size))
            hidden_state = np.zeros((X_batch.shape[0], self.hidden_size))
        else:
            cell_state, hidden_state = initial_states
            
        #X_batch.shape = (num_steps, batch_size, emb_size)
        #Need to transform this somewhere else in LSTM class
        outputs = []
        for timestamp in range(X_batch.shape[0]):
            cell_state, hidden_state = self.LSTMCell(X_batch[timestamp], cell_state, hidden_state)
            outputs.append(hidden_state)
            
        return torch.stack(outputs), (cell_state, hidden_state)